In [ ]:
import pandas as pd
import os
import shutil

# Définir les chemins pour les dossiers des fichiers sources, traités et vides
folder_path = '/home/patea/code/Pateaanania/historique_lit'
processed_folder = '/home/patea/code/Pateaanania/processed'
empty_files_folder = '/home/patea/code/Pateaanania/empty_files'

# Créer les dossiers s'ils n'existent pas
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)
if not os.path.exists(empty_files_folder):
    os.makedirs(empty_files_folder)

# Initialiser une liste pour stocker les DataFrames
dataframes = []

def detect_encoding(file_path):
    import chardet
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read(1024))  # Limiter la lecture à un petit échantillon pour performance
    return result['encoding']

# Parcourir tous les fichiers CSV dans le dossier source
if os.listdir(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            if os.stat(file_path).st_size > 0:
                encoding = detect_encoding(file_path)  # Détecter l'encodage
                try:
                    # Lire le fichier CSV avec l'encodage détecté
                    df = pd.read_csv(file_path, encoding=encoding, sep=";", on_bad_lines='skip')
                    if not df.empty:
                        dataframes.append(df)  # Ajouter le DataFrame à la liste s'il n'est pas vide
                    shutil.move(file_path, os.path.join(processed_folder, file_name))  # Déplacer les fichiers traités
                except Exception as e:
                    print(f"Erreur lors de la lecture du fichier {file_name} avec encodage {encoding}: {e}")
            else:
                print(f"Le fichier {file_name} est vide.")
                shutil.move(file_path, os.path.join(empty_files_folder, file_name))  # Déplacer les fichiers vides
else:
    print("Le dossier est vide.")

# Concaténer tous les DataFrames et créer un fichier Parquet
if dataframes:
    final_dataframe = pd.concat(dataframes, ignore_index=True)
    output_parquet = 'resultat_concatenation_avec_date_heure.parquet'
    final_dataframe.to_parquet(output_parquet, index=False)
    print(f"Le fichier Parquet a été créé avec succès : {output_parquet}")
else:
    print("Aucun fichier CSV valide trouvé ou le dossier est vide.")
